In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pgn import *

model = PointerGenerator()

attn_weights = torch.randn(3, 5, 10)
mem = torch.randn(3, 10, 512)
input_tgt = torch.randn(3, 5, 512)
output_tgt = torch.randn(3, 5, 512)

x = model._compute_p_ins(attn_weights, mem, input_tgt, output_tgt)
x.shape

In [ ]:
import torch
from trans import TransformerDecoderLayer, TransformerDecoder

decoder = TransformerDecoder(
    decoder_layer=TransformerDecoderLayer(512, 8, 2048, 0.1, 1e-5),
    num_layers=3
)

tgt = torch.randn(3, 10, 512)
mem = torch.randn(3, 10, 512)
src_pad_mask = torch.full((3, 10), True)
src_pad_mask[:, :5] = False

x, (*_, attn_weights) = decoder.forward(tgt, mem, memory_key_padding_mask=src_pad_mask)

## num stability

In [ ]:
from pgn import PointerGenerator
model = PointerGenerator(N=64)

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_loader = model._train_loader('data/ud/ud_train_3.0.jsonl', tokenizer, batch_size=4)
eval_loader = model._eval_loader('data/ud/ud_dev_3.0.jsonl', tokenizer, batch_size=4)

In [ ]:
from torch.optim import AdamW
optim = AdamW(model.parameters(), lr=3e-4)
batch = next(iter(train_loader))

model.on_train_epoch_start()

import torch
with torch.autograd.detect_anomaly():
    for i, batch in enumerate(train_loader):
        loss = model.training_step(batch, 0)
        optim.zero_grad()
        loss.backward()
        optim.step()

In [ ]:
import pytorch_lightning as pl

trainer = pl.Trainer(
    max_epochs=2,
    limit_val_batches=10,
    val_check_interval=50,
    check_val_every_n_epoch=None,
    accumulate_grad_batches=1,
    logger=False,
)

trainer.fit(model, train_loader, eval_loader)